In [35]:
import pandas as pd
import numpy as np
from config import datasetPath
from os import path

In [36]:
ratingsPath = path.join(datasetPath, "ratings.csv")

In [37]:
userInteractionData = pd.read_csv(ratingsPath)
# drop timestamp feature
userInteractionData.drop("timestamp", axis=1, inplace=True)

In [38]:
numUsers = len(set(userInteractionData['userId']))
numMovies = len(set(userInteractionData['movieId']))
print("Users:", numUsers)
print("Movies:", numMovies)

Users: 138493
Movies: 26744


In [39]:
# We only count the interaction in the interaction matrix if rating >= ratingCutoff
ratingCutoff = 3
# filter out sparse data
numRows = len(userInteractionData)
userInteractionData.drop(userInteractionData.index[userInteractionData['rating'] < ratingCutoff], axis=0, inplace=True)
# interaction_matrix = np.zeros((numUsers, numMovies))
newNumRows = len(userInteractionData)
print(f"Filtered out {numRows - newNumRows}/{numRows} ({100 * (numRows - newNumRows) / numRows}%) through the rating cuttoff of {ratingCutoff}.")

Filtered out 3513504/20000263 (17.56728899014978%) through the rating cuttoff of 3.


In [40]:
freqCutoff = 5
# if a user has less than freqCutoff interactions, remove user.
numRows = len(userInteractionData)
userInteractionData['user_freq'] = userInteractionData.groupby('userId')['userId'].transform('count')
userInteractionData.drop(userInteractionData.index[userInteractionData['user_freq'] <= freqCutoff], inplace=True)
userInteractionData.drop('user_freq', axis=1)
# if a movie has less than freqCutoff interactions, remove movie.
userInteractionData['movie_freq'] = userInteractionData.groupby('movieId')['movieId'].transform('count')
userInteractionData.drop(userInteractionData.index[userInteractionData['movie_freq'] <= freqCutoff], inplace=True)
userInteractionData.drop('movie_freq', axis=1)
newNumRows = len(userInteractionData)
print(f"Filtered out {numRows - newNumRows}/{numRows} ({100 * (numRows - newNumRows) / numRows}%) through the frequency cuttoff of {freqCutoff}.")

Filtered out 20097/16486759 (0.12189782115453983%) through the frequency cuttoff of 5.


In [41]:
numUsers = len(set(userInteractionData['userId']))
numMovies = len(set(userInteractionData['movieId']))
print("Users:", numUsers)
print("Movies:", numMovies)

Users: 138181
Movies: 16133


In [42]:
interactionMatrix = np.zeros((numUsers, numMovies), dtype=bool)

MemoryError: Unable to allocate 16.6 GiB for an array with shape (138181, 16133) and data type float64